In [1]:
import pandas as pd
from pathlib import Path

ROOT = Path.cwd().parent if (Path.cwd().name == "notebooks") else Path.cwd()

CLEAN_PATH = ROOT / "data" / "clean" / "wps-index-2025-analysis.csv"
LOOKUP_PATH = ROOT / "data" / "external" / "common_countries_data.csv"
OUT_PATH = ROOT / "results" / "wps_joined.csv"

CLEAN_PATH, LOOKUP_PATH, OUT_PATH


(WindowsPath('c:/Users/aisen/OneDrive/Документы/wps-index-2025-analysis/data/clean/wps-index-2025-analysis.csv'),
 WindowsPath('c:/Users/aisen/OneDrive/Документы/wps-index-2025-analysis/data/external/common_countries_data.csv'),
 WindowsPath('c:/Users/aisen/OneDrive/Документы/wps-index-2025-analysis/results/wps_joined.csv'))

In [2]:
wps = pd.read_csv(CLEAN_PATH)
lookup = pd.read_csv(LOOKUP_PATH)

wps.shape, lookup.shape


((181, 18), (181, 6))

In [6]:
wps["Country"] = (
    wps["Country"]
    .astype(str)
    .str.strip()
)

lookup["Country"] = (
    lookup["Country"]
    .astype(str)
    .str.strip()
)

In [7]:
merged = wps.merge(
    lookup,
    how="left",
    on="Country",
    validate="m:1",
    indicator=True
)

merged["_merge"].value_counts(dropna=False)


_merge
both          181
left_only       0
right_only      0
Name: count, dtype: int64

In [9]:
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
merged.to_csv(OUT_PATH, index=False)


# Join Iteration and Missing Values

I initially created a country-level lookup dataset with approximately 100 countries, while the cleaned WPS Index dataset contained 181 countries.

When performing a left merge, countries missing from the lookup resulted in NaN values in the added columns, which demonstrated how incomplete reference data produces missing values during joins.

I decided to expand the lookup dataset to include all countries present in the WPS Index. After aligning coverage, all 181 observations matched successfully, confirming a consistent join key and completeness.

# Why I Chose a Left Merge

I chose a left merge because the WPS Index dataset is the primary dataset and all of its observations should be preserved. The secondary dataset serves only as enrichment, so missing matches should result in NaN values rather than dropping WPS records.